# Helpful Notes

## High-level Task Overview

1. Display a map of current weather events in the US
2. Allow the user to draw a bounding box including some weather events
3. Process the selected weather events to obtain information on the type of event
4. Utilize this information to filter the public Twitter stream
5. Filter the public tweets to this bounded region and the weather event and display the result

## Displaying a map of weather events

We will use the ipyleaflet library (https://ipyleaflet.readthedocs.io/en/latest/index.html) to display a map of weather events and enable controls such as drawing a bounding box.

```python
from ipyleaflet import *

wfs_url = 'https://idpgis.ncep.noaa.gov/arcgis/services/NWS_Forecasts_Guidance_Warnings/watch_warn_adv/MapServer/WFSServer'

m = Map(center=(39.82, -98.58), zoom=4, basemap=basemaps.OpenStreetMap.Mapnik)
wms = WMSLayer(url='http://idpgis.ncep.noaa.gov/arcgis/services/NWS_Forecasts_Guidance_Warnings/watch_warn_adv/MapServer/WMSServer', 
               layers='0',
               format='image/png',
               transparent=True)
m.add_layer(wms)
mymark = Marker(location=[38,-95])
m.add_layer(mymark)
dc = DrawControl()
dc.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "transparent": True
    }
}
dc.on_draw(handle_draw)
m.add_control(dc)
m
```


## Processing a new Bounding Box

Notice that the **on_draw** event utilizes the **handle_draw** event handler. This is where you will need to process the bounding box that was drawn to obtain the relevant information for the selected weather events. 

The NWS WFS service will return information in XML about all events in a specified region. Process the returned XML to obtain details of the weather events.

```python

def handle_draw(self, action, geo_json):
    coords = geo_json['geometry']['coordinates'][0]
    bbox = '%s,%s,%s,%s' % (coords[0][0],coords[0][1],coords[2][0],coords[2][1])
    payload = {'service':'WFS',
               'version': '1.1.0',
               'request': 'GetFeature',
               'typename': 'watch_warn_adv:WatchesWarnings',
               'srsname': 'EPSG:4326',
               'bbox': bbox
              }
    r = requests.get(wfs_url,params=payload)
```

Python's ElementTree (https://docs.python.org/3.6/library/xml.etree.elementtree.html) module is your friend here. Utilize it to process the returned XML (**r.text**). You may need to use Python nltk (https://www.nltk.org/) to break the weather alert sentence into individual keywords.


## Twitter Public Stream

You will need to use Python tweepy (http://docs.tweepy.org/en/v3.5.0/streaming_how_to.html) to access the Twitter public stream. A developer application has been created on Twitter for your use. You will need four pieces of authorization keys from this application:

```python
consumer_key = 'NqYmN1yKycA0iXNggwtee5sze'
consumer_secret = 'NKG9FqJ1SYGeuC6IIitQTmjpPjFZ2bJPn1v9aTjOph7xnD8B6v'
access_token_key = '756563398284935170-Ks0kvwIEVc14v8lkyWN4tgHh8DHRLuY'
access_token_secret = 'Dg6E8vSbHqpjxKzzP4fF1EOBRUiEUkQHdxojnO99SuCAF'
auth = OAuthHandler(ckey, consumer_secret) #OAuth object'
auth.set_access_token(access_token_key, access_token_secret)'
api = API(auth)'
```
